# Diet problem
(Adapted from the pyomo/examples/pyomo files)

The goal of this problem is to *minimize* cost.  
The variables of the problem include the decision of weather a certain food is bought or not.  
Furthermore, the constraints ensure that *at least* one entree, side and drink are selected.

This model is written with the following structure:  

1. Package imports
2. Model and Set declaration
3. Parameter declaration and initialization
4. Variable declaration and initialization
5. Constraint declaration and initialization
6. Objective function
7. Solution

### 1. Package imports

In [1]:
from pyomo.environ import *   #: Same as usual
from pyomo.opt import SolverFactory

### 2. Model and Set declaration

Note how the model *must* be declared first. Subsequently, every component from the model is attached to the model.

In [2]:
m = ConcreteModel()  #: Declare model

The set *food* contains the overall available dishes.  
In order to create this set, there are a number of ways to specify its items.   
*One* of the possible ways is to specify its items as a *python list* of strings, i.e.:

In [3]:
foods = ["QPwCheese", "MDwCheese", "LeBigMac", "FOFish", "McGChicken", "Fries", "McSausage", "LfMilk", "OJ"]

Then, the set is *initialized* using the `initialize` keyword of the Set constructor,

In [4]:
m.food = Set(initialize=foods)

Finally, we can reuse the dictionary to define the kinds of dishes available,

In [5]:
entree = foods[:5]
side = foods[5:7]
drink = foods[7:]
print("entres: ", entree)
print("sides: ", side)
print("drinks: ", drink)

entres:  ['QPwCheese', 'MDwCheese', 'LeBigMac', 'FOFish', 'McGChicken']
sides:  ['Fries', 'McSausage']
drinks:  ['LfMilk', 'OJ']


In [6]:
NUTR = ["Cal", "Carbo", "Protein", "VitA", "VitC", "Calc", "Iron"]

In [7]:
m.nutr = Set(initialize=NUTR)

### 3. Parameter declaration and initialization

The parameters are declared over a set (e.g. food). To specify the values of such parameters, it is possible to use a dictionary whose key correspond to the values of the set.

In [8]:
COST_DICT = dict.fromkeys(foods)
COST_DICT = {"QPwCheese": 1.84,
             "MDwCheese": 2.19, 
             "LeBigMac": 1.84, 
             "FOFish": 1.44, 
             "McGChicken": 2.29, 
             "Fries": 0.77, 
             "McSausage": 1.29, 
             "LfMilk": 0.60, 
             "OJ": 0.72}

Then, the dictionary is passed to the respective parameter using the `initialize` keyword.

In [9]:
m.cost = Param(m.food, initialize=COST_DICT, within=PositiveReals)

Another way to initialize parameters over a set is using the `default` keyword.

In [28]:
m.f_min = Param(m.food, within=NonNegativeReals, default=300.0)

    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [11]:
MAX_FOOD_SUPPLY = 20.0

In [29]:
m.f_max = Param(m.food, default=MAX_FOOD_SUPPLY)

    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
    usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [14]:
N_MIN = {"Cal": 2000, "Carbo": 350, "Protein": 55, "VitA": 100, "VitC": 100, "Calc": 100, "Iron": 100}

In [15]:
m.n_min = Param(m.nutr, initialize=N_MIN)

In [16]:
N_MAX = {"Cal": 0, "Carbo": 375, "Protein": 0, "VitA": 0, "VitC": 0, "Calc": 0, "Iron": 0}

In [17]:
m.n_max = Param(m.nutr, initialize=N_MAX)

In [18]:
AMT = {}
AMT[("QPwCheese","Cal")] = 510
AMT[("MDwCheese","Cal")] = 370
AMT[("LeBigMac","Cal")] = 500
AMT[("FOFish","Cal")] = 370
AMT[("McGChicken","Cal")] = 400
AMT[("Fries","Cal")] = 220
AMT[("McSausage","Cal")] = 345
AMT[("LfMilk","Cal")] = 110
AMT[("OJ","Cal")] = 80
AMT[("QPwCheese","Carbo")] = 34
AMT[("MDwCheese","Carbo")] = 35
AMT[("LeBigMac","Carbo")] = 42
AMT[("FOFish","Carbo")] = 38
AMT[("McGChicken","Carbo")] = 42
AMT[("Fries","Carbo")] = 26
AMT[("McSausage","Carbo")] = 27
AMT[("LfMilk","Carbo")] = 12
AMT[("OJ","Carbo")] = 20
AMT[("QPwCheese","Protein")] = 28
AMT[("MDwCheese","Protein")] = 24
AMT[("LeBigMac","Protein")] = 25
AMT[("FOFish","Protein")] = 14
AMT[("McGChicken","Protein")] = 31
AMT[("Fries","Protein")] = 3
AMT[("McSausage","Protein")] = 15
AMT[("LfMilk","Protein")] = 9
AMT[("OJ","Protein")] = 1
AMT[("QPwCheese","VitA")] = 15
AMT[("MDwCheese","VitA")] = 15
AMT[("LeBigMac","VitA")] = 6
AMT[("FOFish","VitA")] = 2
AMT[("McGChicken","VitA")] = 8
AMT[("Fries","VitA")] = 0
AMT[("McSausage","VitA")] = 4
AMT[("LfMilk","VitA")] = 10
AMT[("OJ","VitA")] = 2
AMT[("QPwCheese","VitC")] = 6
AMT[("MDwCheese","VitC")] = 10
AMT[("LeBigMac","VitC")] = 2
AMT[("FOFish","VitC")] = 0
AMT[("McGChicken","VitC")] = 15
AMT[("Fries","VitC")] = 15
AMT[("McSausage","VitC")] = 0
AMT[("LfMilk","VitC")] = 4
AMT[("OJ","VitC")] = 120
AMT[("QPwCheese","Calc")] = 30
AMT[("MDwCheese","Calc")] = 20
AMT[("LeBigMac","Calc")] = 25
AMT[("FOFish","Calc")] = 15
AMT[("McGChicken","Calc")] = 15
AMT[("Fries","Calc")] = 0
AMT[("McSausage","Calc")] = 20
AMT[("LfMilk","Calc")] = 30
AMT[("OJ","Calc")] = 2
AMT[("QPwCheese","Iron")] = 20
AMT[("MDwCheese","Iron")] = 20
AMT[("LeBigMac","Iron")] = 20
AMT[("FOFish","Iron")] = 10
AMT[("McGChicken","Iron")] = 8
AMT[("Fries","Iron")] = 2
AMT[("McSausage","Iron")] = 15
AMT[("LfMilk","Iron")] = 0
AMT[("OJ","Iron")] = 2

In [19]:
m.amt = Param(m.food, m.nutr, initialize=AMT)

A third way is to use a python function, and setting the keyword to the function name (`initialize=function_name`).

## 4. Variables

In [20]:
def buy_bounds(mod, i):
    return (mod.f_min[i], mod.f_max[i])

m.buy = Var(m.food, bounds=buy_bounds, within=NonNegativeIntegers)

## 5. Constraints

In [22]:
def entree_rule(mod):
    return sum(mod.buy[e] for e in entree) >= 1

m.entree = Constraint(rule=entree_rule)

In [23]:
def side_rule(mod):
    return sum(mod.buy[s] for s in side) >= 1

m.side = Constraint(rule=side_rule)

In [24]:
def drink_rule(mod):
    return sum(mod.buy[d] for d in drink) >=1

m.drink = Constraint(rule=drink_rule)

## 6. Objective

In [21]:
def total_cost_rule(mod):
    return sum(mod.cost[j] * mod.buy[j] for j in mod.food)
m.total_cost = Objective(rule=total_cost_rule, sense=minimize)

## 7. Solution

In [25]:
opt = SolverFactory('cbc')  #: declare the solver

In [26]:
results = opt.solve(m, tee=True)

    cbc


ApplicationError: No executable found for solver 'cbc'